# Notebook details

In [ ]:
def setup_notebook(fix_python_path=True, reduce_margins=True, plot_inline=True):
    if reduce_margins:
        # Reduce side margins of the notebook
        from IPython.core.display import display, HTML
        display(HTML("<style>.container { width:100% !important; }</style>"))

    if fix_python_path:
        # add egosocial to the python path
        import os, sys
        sys.path.extend([os.path.dirname(os.path.abspath('.'))])

    if plot_inline:
        # Plots inside cells
        %matplotlib inline
    
    global __file__
    __file__ = 'Notebook'

setup_notebook()

# Imports and Constants Definition

In [ ]:
# !/usr/bin/env python
# -*- coding: utf-8 -*-

import argparse
from collections import defaultdict
import functools
import itertools
import json
import logging
import os
import pprint
import sys

from IPython.display import SVG

import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import pandas as pd
import sklearn
import scipy

from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import LeavePGroupsOut
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.utils import compute_class_weight

import keras
from keras import backend as K
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.utils.vis_utils import model_to_dot

import egosocial
import egosocial.config
from egosocial.core.attributes import AttributeSelector
from egosocial.core.models import create_model_top_down
from egosocial.core.models import create_model_bottom_up
from egosocial.utils.filesystem import create_directory 
from egosocial.utils.filesystem import check_directory
from egosocial.utils.keras.autolosses import AutoMultiLossWrapper
from egosocial.utils.keras.backend import limit_gpu_allocation_tensorflow
from egosocial.utils.keras.callbacks import PlotLearning
from egosocial.utils.keras.metrics import precision
from egosocial.utils.keras.metrics import recall
from egosocial.utils.keras.metrics import fmeasure
from egosocial.utils.keras.processing import TimeSeriesDataGenerator
from egosocial.utils.logging import setup_logging
from egosocial.utils.sklearn.model_selection import StratifiedGroupShuffleSplitWrapper
from egosocial.utils.misc import RELATIONS, DOMAINS
from egosocial.utils.misc import LabelExpander
from egosocial.utils.misc import decode_prediction
from egosocial.utils.misc import relation_to_domain_vec

SHARED_SEED = 42

# Limit GPU memory allocation with Tensorflow

In [ ]:
limit_memory = True
if limit_memory and K.backend() == 'tensorflow':
    memory_ratio = 0.3
    limit_gpu_allocation_tensorflow(memory_ratio)

# Fake main

In [ ]:
 def main(*fake_args):
    entry_msg = 'Train model for social relations classification in egosocial photo-streams.'
    parser = argparse.ArgumentParser(description=entry_msg)

    parser.add_argument('--dataset_path', required=True,
                        help='Path to file containing the input data and labels information merged.')

    parser.add_argument('--features_dir', required=True,
                        help='Directory where the extracted features are stored.')
    
    parser.add_argument('--reuse_model', required=False,
                        action='store_true',
                        help='Use precomputed model if available.')

    parser.add_argument('--save_model', required=False,
                        action='store_true',
                        help='Save model to disk.')

    parser.add_argument('--save_stats', required=False,
                        action='store_true',
                        help='Save statistics to disk.')

    parser.add_argument('--epochs', required=False, type=int,
                        default=30,
                        help='Max number of epochs.')

    parser.add_argument('--batch_size', required=False, type=int,
                        default=32,
                        help='Batch size.')
    
    parser.add_argument('--lr', required=False, type=float,
                        default=0.001,
                        help='Initial learning rate.')
    
    if not os.path.isdir(egosocial.config.TMP_DIR):
        os.mkdir(egosocial.config.TMP_DIR)

    setup_logging(egosocial.config.LOGGING_CONFIG,
                  log_dir=egosocial.config.LOGS_DIR)
    
    # TODO: implement correctly
    args = parser.parse_args(*fake_args)

    return args

# Helper functions

In [ ]:
def parse_day(image_path):
    image_name = os.path.basename(image_path)
    # a valid image follows the day_hour_x.ext format
    day_hour_rest = image_name.split('_')
    
    if len(day_hour_rest) == 3:
        # day is the first item
        return day_hour_rest[0]
    else:
        # day isn't available
        return ''
    
def load_dataset_defition(dataset_path, include_day=True):
    with open(dataset_path, 'r') as json_file:
        dataset_def = json.load(json_file)

    # flatten the segments structure
    samples = pd.DataFrame(list(itertools.chain(*dataset_def)))
    
    if include_day:
        samples['day'] = samples['global_image_path'].apply(parse_day)
    
    return samples

def load_features(features_path, data_frames):
    features = np.load(features_path)
    sequences_info = data_frames.groupby(['split', 'segment_id', 'group_id'])    

    feature_sequences = []
    for seq_id, group in sequences_info:
        feature_seq = features[group.index]
        feature_seq.shape = (feature_seq.shape[0], -1)
        feature_sequences.append(feature_seq)
    
    return np.asarray(feature_sequences)

def load_fields(data_frames, fields, valid_frames_idx=None):
    assert len(fields) > 0
    
    if valid_frames_idx is None:
        sequences_info = data_frames.groupby(['split', 'segment_id', 'group_id'])
    else:
        sequences_info = data_frames[valid_frames_idx].groupby(['split', 'segment_id', 'group_id'])
    
    fst_seq_frames = [group.index[0] for _, group in sequences_info]
    fields_data =  data_frames.iloc[fst_seq_frames][fields].values

    return [fields_data[:, field_idx] for field_idx in range(len(fields))]
        
def compute_stats(y, y_predicted):
    acc = sklearn.metrics.accuracy_score(y, y_predicted)
    confusion_matrix = sklearn.metrics.confusion_matrix(y, y_predicted)
    report = sklearn.metrics.classification_report(y, y_predicted)

    return acc, confusion_matrix, report

def print_statistics(val_stats=None, test_stats=None, fdesc=sys.stdout):
    for description, stats in [('Validation set:', val_stats), (('Test set:', test_stats))]:
        
        if stats is not None:
            print(description, file=fdesc)
            accuracy, confusion_matrix, report = stats
            print('Confusion matrix:', file=fdesc)
            print(confusion_matrix, file=fdesc)
            print(file=fdesc)
            print(report, file=fdesc)
            print('Accuracy: {:.3f}'.format(accuracy), file=fdesc)
            print('------------------------------------------------', file=fdesc)
            
def compute_class_frequency(y, index=None):
    if index:
        y =  y[index]
    _, counts = np.unique(y, return_counts=True)
    freq = counts / len(y)
    return freq

def compute_class_weight_labels(y, mode=None):
    if mode == 'both_fused':
        raise NotImplementedError('Class weights are not available in mode both_fused.')

    encoder = LabelExpander(mode=mode)
    y = encoder(y)

    class_weight = {}
    for y_type, y_data in y.items():
        data = list(np.argmax(y_data, axis=1))    
        classes = np.unique(data)
        weights = compute_class_weight('balanced', classes, data)
        class_weight[y_type] = dict(zip(classes, weights))
    
    return class_weight
                
class DimReductionTransformer(object):        

    def __init__(self, n_components, Q=32, normalize=True, random_state=None):
        # PCA configuration (number of components or min explained variance)
        self.pca_param = n_components
         # features quantization (smaller Q promotes sparsity)
        self.Q = Q
        self.normalize = normalize
        self.random_state = sklearn.utils.check_random_state(random_state)
        
        self._scaler = None
        self._pca = None
        
        self._log = logging.getLogger(self.__class__.__name__)
        
    def fit(self, x):
        # reset state
        self._scaler = None
        self._pca = None        
    
        if self.normalize:
            if self.Q: # quantization requires data in range [0, 1] 
                self._scaler = Normalizer(norm='l2')
            else:
                self._scaler = StandardScaler()
            
            self._log.debug('Fitting data normalization.')            
            x = self._scaler.fit_transform(x)            
            
        if self.Q:
            # small Q promotes sparsity
            x = np.floor(self.Q * x)

        self._log.debug('Fitting PCA.')        
        assert self.pca_param > 0     
        # compute pca from scratch
        if 0 < self.pca_param <= 1:
            # running pca with min explained variance takes much longer
            self._pca = PCA(self.pca_param, random_state=self.random_state)
        else:
            self._pca = PCA(self.pca_param, svd_solver='randomized', random_state=self.random_state)
        
        self._pca.fit(x)
        
    def transform(self, x):
        if self.normalize:
            self._log.debug('Applying data normalization')
            x = self._scaler.transform(x)

        if self.Q:
            # small Q promotes sparsity
            self._log.debug('Applying Q-sparsity Q={}'.format(self.Q))
            x = np.floor(self.Q * x)            

        explained_var = np.sum(self._pca.explained_variance_ratio_)
        n_components = self._pca.n_components_
        msg = 'Applying PCA with explained var {} dims {}'
        self._log.debug(msg.format(explained_var, n_components))
        # pca transformation
        x = self._pca.transform(x)

        return x

In [ ]:
class SocialClassifierWithPreComputedFeatures:
    
    def __init__(self, dataset_path, features_dir, test_size=0.2, k_fold_splits=10, val_size=None, n_components=50, Q=32, seed=42):
        self.dataset_path = dataset_path
        self.features_dir = features_dir
        self.test_size = test_size
        self.k_fold_splits = k_fold_splits
        self.val_size = val_size if val_size else 1.0 / k_fold_splits
        self.n_components = n_components
        self.Q = Q
        self.seed = seed

        self._frames = None
        self._grouped_frames = None
        self._labels = None
        self._users = None
        self._attribute_features = None
        self._train_idx = None
        self._test_idx = None
        self._k_train_val_idx = None
        
        self._log = logging.getLogger(self.__class__.__name__)

    def load_data(self):        
        # load dataset definition
        frames = load_dataset_defition(self.dataset_path, include_day=True)
        # filter labels with few samples
        valid_frames_idx = np.isin(frames['relation_label'], RELATIONS)

        self._frames = frames[valid_frames_idx]
        self._grouped_frames = self._frames.groupby(['split', 'segment_id', 'group_id'])

        # for each sequence get label, user, day of first frame
        self._labels, self._users, self._seq_days = load_fields(
            frames, ['relation_label', 'camera_user_name', 'day'], 
            valid_frames_idx=valid_frames_idx
        )
        
        self._init_features(self._frames)
        self._init_grouped_splits()
        self._dim_reduction = defaultdict(dict)

    def _init_features(self, valid_frames):
        attributes = sorted(next(os.walk(self.features_dir))[2])
        attributes = [attr for attr in attributes if attr.endswith('.npy')]

        # load features
        self._attribute_features = {}
        for attribute_file in attributes:
            attribute_name, ext = os.path.splitext(attribute_file)
            features_path = os.path.join(self.features_dir, attribute_file)
            self._attribute_features[attribute_name] = load_features(features_path, valid_frames)        
        
    def _init_stratified_splits(self):
        # define data splits
        # define train, test splits
        sss = StratifiedShuffleSplit(test_size=self.test_size, random_state=self.seed)
        y = self._labels
        self._train_idx, self._test_idx = next(sss.split(np.zeros(len(y)), y))  

        # define k-fold train splits
        skf = StratifiedKFold(n_splits=self.k_fold_splits, random_state=self.seed)
        y = np.array(y[self._train_idx])
        
        self._k_train_val_idx = []
        for t_idx, v_idx in skf.split(np.zeros(len(y)), y):
            self._k_train_val_idx.append((self._train_idx[t_idx], self._train_idx[v_idx]))

    def _init_grouped_splits(self):
        # define data splits
        # define train, test splits        
        criteria = np.array([ user + '_' + day for user, day in zip(self._users, self._seq_days) ])
        y, groups = helper._labels, criteria

        n_tries, group_size, epsilon = 1000, self.test_size, 0.025

        split_wrapper = StratifiedGroupShuffleSplitWrapper(
            GroupShuffleSplit(n_splits=n_tries, test_size=group_size, random_state=self.seed), 
            n_splits=1, 
            max_test_size=min(self.test_size + epsilon, 1.0), min_test_size=max(self.test_size - epsilon, 0.0)
        )
        self._train_idx, self._test_idx, train_test_score = next(split_wrapper.split(np.zeros(len(y)), y, groups, return_score=True))
        test_size = len(self._test_idx) / (len(self._train_idx) + len(self._test_idx))
        self._log.debug('Split train-test score: {:.3} real_test_size: {:.3}'.format(train_test_score, test_size))
        
        # define k-fold splits
        y, groups = y[self._train_idx], groups[self._train_idx]
        
        if self.k_fold_splits > 1:
            # k-fold strategy
            # search 50 times the number of splits, encourage diversity
            # double the epsilon (more flexible)
            n_tries, group_size, epsilon = self.k_fold_splits * 50, self.val_size, 0.05
        else:
            # holdout strategy
            # keep n_tries and epsilon same as train-test split
            group_size = self.val_size
        
        split_wrapper = StratifiedGroupShuffleSplitWrapper(
            GroupShuffleSplit(n_splits=n_tries, test_size=group_size, random_state=self.seed), 
            n_splits=self.k_fold_splits,
            max_test_size=min(self.val_size + epsilon, 1.0), min_test_size=max(self.val_size - epsilon, 0.0)
        )
        
        self._k_train_val_idx = []
        for k, (t_idx, v_idx, t_v_score) in enumerate(split_wrapper.split(np.zeros(len(y)), y, groups, return_score=True)):
            self._k_train_val_idx.append((self._train_idx[t_idx], self._train_idx[v_idx]))
            val_size = (1 - test_size) * len(v_idx) / (len(t_idx) + len(v_idx))
            self._log.debug('{}-fold split score: {:.3} real_val_size={:.3}'.format(k, t_v_score, val_size))            
            
    def list_attributes(self):
        # list attributes
        if self._attribute_features:
            return sorted(self._attribute_features.keys())
        else:
            return []
            
    def _get_split_idx(self, split, k_fold=None):
        assert split in ('train', 'test', 'val')
        if split == 'train':
            if k_fold is None:
                return self._train_idx
            else:
                assert 0 <= k_fold < self.k_fold_splits
                return self._k_train_val_idx[k_fold][0]

        if split == 'val':
            assert 0 <= k_fold < self.k_fold_splits
            return self._k_train_val_idx[k_fold][1]
        
        if split == 'test':
            return self._test_idx
        
    def get_k_train_val_split(self, selected_attributes, k_fold, preprocess_mask=None):
        assert 0 <= k_fold < self.k_fold_splits
        return self._get_train_test_val_split(selected_attributes, k_fold=k_fold, preprocess_mask=preprocess_mask)
        
    def get_train_test_split(self, selected_attributes, preprocess_mask=None):
        return self._get_train_test_val_split(selected_attributes, preprocess_mask=preprocess_mask)
    
    def _get_train_test_val_split(self, selected_attributes, k_fold=None, preprocess_mask=None):
        # get data splits composed by selected attributes only
        # preprocess the data

        # preconditions
        assert self._attribute_features is not None
        assert self._labels is not None
        assert len(selected_attributes) > 0 
        assert preprocess_mask is None or len(preprocess_mask) == len(selected_attributes)
        min_dim_for_pca = self.n_components
        
        if k_fold is None:
            train_idx, test_idx = self._get_split_idx('train'), self._get_split_idx('test')
            k_fold = -1 # see DimReductionTransformer init below
        else:
            assert 0 <= k_fold < self.k_fold_splits
            train_idx, test_idx = self._get_split_idx('train', k_fold), self._get_split_idx('val', k_fold)
            
        # preprocess each selected attribute individually
        train_features_list, test_features_list = [], []
        for attr_i, attr in enumerate(selected_attributes):            
            train_features = self._attribute_features[attr][train_idx]
            train_features = list(itertools.chain(*train_features))
            
            test_features = self._attribute_features[attr][test_idx]
            test_features = list(itertools.chain(*test_features))
            
            n_features = train_features[0].shape[-1]
            if preprocess_mask and preprocess_mask[attr_i]:
                self._log.debug('Preprocessing {}.'.format(attr))
                # initialize transformation for each attribute
                if min_dim_for_pca <= n_features:
                    # init transformation with different (fixed) seed for each fold
                    transformation = DimReductionTransformer(n_components=self.n_components, Q=self.Q, random_state=self.seed + k_fold)
                else:
                    transformation = StandardScaler()

                transformation.fit(train_features)
            
                train_features = transformation.transform(train_features)
                test_features = transformation.transform(test_features)
            else:
                self._log.debug('Skip preprocessing {}.'.format(attr))

            train_features_list.append(train_features)
            test_features_list.append(test_features)

        attr = selected_attributes[0]
        # create sequence structure for train
        train_features_fused = np.concatenate(train_features_list, axis=-1)
        train_seq_len = map(len, self._attribute_features[attr][train_idx])
        train_features_as_seqs = []
        offset = 0
        for seq_len in train_seq_len:        
            seq_range = range(offset, offset+seq_len)
            train_features_as_seqs.append(train_features_fused[seq_range])
            offset += seq_len
        
        # create sequence structure for testtrain
        test_features_fused = np.concatenate(test_features_list, axis=-1)
        test_seq_len = map(len, self._attribute_features[attr][test_idx])
        test_features_as_seqs = []
        offset = 0
        for seq_len in test_seq_len:        
            seq_range = range(offset, offset+seq_len)
            test_features_as_seqs.append(test_features_fused[seq_range])
            offset += seq_len
        
        return (train_features_as_seqs, self._labels[train_idx]), (test_features_as_seqs, self._labels[test_idx])
    
    def max_sequence_len(self):
        return self._grouped_frames.size().max()

# Configure Keras

In [ ]:
def init_callbacks(output_mode, plot_stats=True, save_model=False, save_stats=False, stop_early=False, reduce_lr=False, figsize=None):
    callbacks = []

    training_dir = os.path.join(egosocial.config.TMP_DIR, 'training')
    create_directory(training_dir, 'Training')

    if save_model:
        checkpoint_path = os.path.join(training_dir,
                                       'weights.{epoch:02d}-{val_loss:.2f}.h5')
        checkpointer = ModelCheckpoint( 
            filepath=checkpoint_path, monitor='val_loss',
            save_best_only=True, period=5,
        )
        callbacks.append(checkpointer)

    if save_stats:
        metrics_path = os.path.join(training_dir,
                                    'metrics.csv')
        csv_logger = CSVLogger(metrics_path)
        callbacks.append(csv_logger)

    if reduce_lr:
        lr_handler = ReduceLROnPlateau(
            monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001
        )
        callbacks.append(lr_handler)
   
    if plot_stats:
        # more plots need more space
        if not figsize:
            if output_mode != 'both_splitted':
                figsize = (25, 5)
            else:
                figsize = (25, 13)

        plot_metrics = PlotLearning(update_step=5, figsize=figsize)
        callbacks.append(plot_metrics)
        
    if stop_early:
        stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, mode='auto')
        callbacks.append(stopper)
    
    return callbacks

def compile_model(
    model, 
    optimizer='adam', 
    loss='categorical_crossentropy', 
    loss_weights='auto',
    **kwargs
):
    # wrapper allows to train the loss weights
    model_wrapper = AutoMultiLossWrapper(model)
    model_wrapper.compile(optimizer=optimizer, loss=loss, 
                          loss_weights=loss_weights, **kwargs)

    return model_wrapper.model

In [ ]:
BASE_DIR = os.path.join(egosocial.config.TMP_DIR, 'egocentric', 'datasets')

args = [
    "--dataset_path", os.path.join(BASE_DIR, 'merged_dataset.json'),
    "--features_dir", os.path.join(BASE_DIR, 'extracted_features'),
]

conf = main(args)

# Loading precomputed features and labels

In [ ]:
helper = SocialClassifierWithPreComputedFeatures(
    conf.dataset_path, conf.features_dir, 
    test_size=0.2, 
    k_fold_splits=3, 
    val_size=0.2, 
    n_components=50, 
    Q=32,
    seed=SHARED_SEED
)

helper.load_data()

# Some statistics

In [ ]:
stats = []
for user in np.unique(helper._users):
    user_labels = helper._labels[helper._users == user]
    unique_user_labels = np.unique(user_labels)
    stats.append({'User':user, '# samples': len(user_labels), '# labels:': len(unique_user_labels),  'labels' : unique_user_labels})
stats = pd.DataFrame(stats)
stats

# Select attributes (default all), prepare output

In [ ]:
attribute_selector = AttributeSelector(helper.list_attributes())

# all / face / body / or single attribute (accept name substring, e.g. activity)
attributes_query = 'all'
# expand all / face / body / single attribute
selected_attributes = attribute_selector.filter(attributes_query)
print('Selected attribute(s): {}'.format(selected_attributes))

to_process = lambda attr : attr not in ('camera_user_age', 'camera_user_gender')
mask = [to_process(attr) for attr in selected_attributes]

# Prepare data

In [ ]:
helper.Q = 32
helper.n_components = 50

In [ ]:
k_fold=0
# prepare splits for selected attributes
(k_x_train, k_y_train), (k_x_val, k_y_val) = helper.get_k_train_val_split(selected_attributes, k_fold=k_fold, preprocess_mask=mask)

# Parameters

In [ ]:
n_train, n_val = len(k_x_train), len(k_x_val)
n_features = k_x_train[0].shape[-1]
max_timestep = helper.max_sequence_len()
batch_size = 64

print('Length of the largest sequence:', max_timestep)
print('Number of features:', n_features)
print('Number of samples fold={}. Training: {} / Validation {}'.format(k_fold, n_train, n_val))
print('Batch size: {}'.format(batch_size))

#output_mode = 'both_fused' # domain-relation outputs fused
#output_mode = 'both_splitted' # multi-loss domain-relation
output_mode = 'domain' # domain only
#output_mode = 'relation' # relation only

print('Output mode: {}'.format(output_mode))

In [ ]:
# use domain specific labels
if output_mode == 'domain':
    k_y_train = relation_to_domain_vec(k_y_train)
    k_y_val = relation_to_domain_vec(k_y_val)    
    
# class_weight for keras (balance domain/relation instances)
class_weight = compute_class_weight_labels(k_y_train, mode=output_mode)

if 'domain' in class_weight:
    pprint.pprint(('domains', [(domain, class_weight['domain'][idx]) for idx, domain in enumerate(DOMAINS)]))
    
if 'relation' in class_weight:
    pprint.pprint(('relations', [(relation, class_weight['relation'][idx]) for idx, relation in enumerate(RELATIONS)]))
    
process_labels = LabelExpander(mode=output_mode)

# Model instance

In [ ]:
learning_rate = 0.0002
print('LR: {}'.format(learning_rate))

In [ ]:
model_strategy = 'top_down'
model_strategy_select = {
    'top_down' : create_model_top_down,
    'bottom_up' : create_model_bottom_up,
}

model_parameters = dict(
    max_seq_len=max_timestep,
    n_features=n_features,
    mode=output_mode,
    units=96,
    recurrent_type='GRU',
    drop_rate=0.5,
    rec_drop_rate=0.5,
    l2_reg=0.01,
    hidden_fc=0,
    n_relations=len(RELATIONS),
    n_domains=len(DOMAINS),  
)
    
model = model_strategy_select[model_strategy](**model_parameters)
if model_strategy_select == 'bottom_up':
    model.get_layer('domain').set_weights([relation_to_domain_weights()])

model = compile_model(
    model,
    loss='categorical_crossentropy',    
    optimizer=keras.optimizers.Adam(learning_rate, decay=1e-5),
    metrics=['accuracy', recall, fmeasure],
    loss_weights='auto',
)

model.summary()

# Training

In [ ]:
callbacks = init_callbacks(output_mode, plot_stats=True, stop_early=True, figsize=(25, 13))

train_datagen = TimeSeriesDataGenerator(fancy_pca=True, noise_stddev=0.01, random_state=SHARED_SEED)
train_datagen.fit(k_x_train)

val_datagen = TimeSeriesDataGenerator()

train_generator = train_datagen.flow(
    k_x_train, k_y_train,
    maxlen=max_timestep,
    output_cbk=process_labels,
    batch_size=batch_size,
    seed=SHARED_SEED,
    shuffle=True,
)

val_batch_size = n_val
val_generator = val_datagen.flow(
    k_x_val, k_y_val,
    maxlen=max_timestep,
    output_cbk=process_labels,
    batch_size=val_batch_size,
    seed=SHARED_SEED,
    shuffle=False,
)

In [ ]:
cache_epochs, epochs = 0, 400
print('Initial epoch: {} Number of epochs: {}'.format(cache_epochs, epochs))

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(1.0 * n_train / batch_size),
    validation_data=val_generator,
    validation_steps=np.ceil(1.0 * n_val / val_batch_size),
    epochs=cache_epochs + epochs,
    initial_epoch=cache_epochs,
    callbacks=callbacks,
    verbose=0,
    max_queue_size=10,
    workers=4,
    class_weight=class_weight,
)

# Evaluation

In [ ]:
val_generator = val_datagen.flow(
    k_x_val, k_y_val, 
    maxlen=max_timestep, 
    output_cbk=process_labels, 
    batch_size=n_val,
    seed=SHARED_SEED,
    shuffle=False,
)

k_y_val_predicted = model.predict_generator(val_generator, steps=1)

decoded_prediction = decode_prediction(k_y_val_predicted, mode=output_mode)

if 'domain' in decoded_prediction:
    print_statistics(val_stats=compute_stats(k_y_val, decoded_prediction['domain']))

if 'relation' in decoded_prediction:
    print_statistics(val_stats=compute_stats(k_y_val,  decoded_prediction['relation']))

# Model definition and graph

In [ ]:
SVG(model_to_dot(model).create(prog='dot', format='svg'))